# Simple Linear Regression

Student Heigth and Weight example

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.power as smp


### Example: Height and weight

Linear regression with height and weight data

In [ ]:
# data
x = np.array([168, 161, 167, 179, 184, 166, 198, 187, 191, 179])             # height data
y = np.array([65.5, 58.3, 68.1, 85.7, 80.5, 63.4, 102.6, 91.4, 86.7, 78.9])  # weight data

Today we have WEIGHT as our outcome variable (y)

and HEIGHT as our explanatory variable (x)

In [ ]:
# ALWAYS start by visualising your data!

# we make a scatter plot:
plt.scatter(x,y)
plt.ylabel("weight")
plt.xlabel("height")
plt.show()

Linear relationship?

Now lets put data into a **pandas dataframe** that we call "student":

In [ ]:
student = pd.DataFrame({'x': x, 'y': y})  # "import pandas as pd"
print(student)

Now we do a **linear regression model**:

In [ ]:
# for now we just run the code, we will understand it later

# fit the model
fitStudents = smf.ols(formula = 'y ~ x', data=student).fit()  # OBS: use the statsmodels.formula.api library (smf)

# Get prediction and confidence intervals
x_pred = pd.DataFrame({'x': np.arange(160,200, 1)})
pred = fitStudents.get_prediction(x_pred).summary_frame(alpha=0.05)

plt.scatter(x,y)
plt.plot(x_pred, pred["mean"], color="red")
plt.ylabel("weight")
plt.xlabel("height")
plt.show()

The red line is the **regression line**.

The regression line is a straight line.

Is is defined by a **slope** and an **intercept** with the y-axis.

In [ ]:
# later today we will be able to estimate values for the parameters: intercept and slope

# we will also learn to make a "Regression Results Table" with Python (using smf library)
print(fitStudents.summary(slim=True))


The table includes a lot of information and we will understand almost all of it by the end of today :)

We will also be able to estimate **standard errors** and **conficence intervals** for the parameters of the model (also in table above)

Later today we will be able to make a **prediction interval** for "future" **observations** (with other x-values)

In [ ]:
# plot of prediction interval
plt.scatter(x,y)
plt.plot(x_pred, pred["mean"], color="red")
plt.plot(x_pred, pred["obs_ci_lower"], color="lightgrey")
plt.plot(x_pred, pred["obs_ci_upper"], color="lightgrey")
plt.ylabel("weight")
plt.xlabel("height")
plt.show()


and later today we will also be able to make a **confidence interval** for the mean at every value on the x-axis (*confidence interval for the line*)

In [ ]:
# add confidence interval for the line
plt.scatter(x,y)
plt.plot(x_pred, pred["mean"], color="red")
plt.plot(x_pred, pred["mean_ci_lower"], color="grey")
plt.plot(x_pred, pred["mean_ci_upper"], color="grey")
plt.plot(x_pred, pred["obs_ci_lower"], color="lightgrey")
plt.plot(x_pred, pred["obs_ci_upper"], color="lightgrey")
plt.ylabel("weight")
plt.xlabel("height")
plt.show()


Other things we will talk about:

1) How does correlation relate to the linear regression (slope)?

2) How much of the total variation in y is explained by the linear relationship?

(back to slides)

### Example: Estimating parameters $\beta_0$ and $\beta_1$

Simulated data:

In [ ]:
# First we compute som simulated data, so we know the "real" beta_0 and beta_1 (and sigma)
np.random.seed(23498)

beta_0 = 50
beta_1 = 200
sigma = 90

# choose som random x-values:
x = stats.uniform.rvs(size = 20, loc=-2, scale = 6)
# simulate y-values from statistical model:
y = beta_0 + beta_1*x + stats.norm.rvs(size = 20, loc=0, scale = sigma)

plt.scatter(x,y)
plt.show()

This is now our "Toy Data" :-)

Lets see if we can estimate the parameters 

In [ ]:
# calculate estimates beta_0_hat and beta_1_hat:

Sxx = np.sum((x - x.mean())**2)
beta_1_hat = np.sum((x - x.mean())*(y - y.mean())) / Sxx
beta_0_hat = y.mean() - beta_1_hat*x.mean()

print(beta_0_hat, beta_1_hat)
print(beta_0, beta_1)

In [ ]:
# plot estimated model with data:
plt.scatter(x,y)
plt.plot([-2,4], [beta_0_hat + beta_1_hat*(-2), beta_0_hat + beta_1_hat*(4)], color="red")
plt.show()


Great! 

also plot the "true" underlying model (this is *only* possible because we have simulated data - IRL one cannot plot the "true" model):

In [ ]:
plt.scatter(x,y)
plt.plot([-2,4], [beta_0_hat + beta_1_hat*(-2), beta_0_hat + beta_1_hat*(4)], color="red")
plt.plot([-2,4], [beta_0 + beta_1*(-2), beta_0 + beta_1*(4)], color="black", linestyle='--')
plt.show()

We can also estimate the variance of residuals:

In [ ]:
res = y - beta_0_hat - beta_1_hat * x
sigma_hat = np.sqrt(np.sum(res**2)/(20-2))
print(sigma_hat)

Compare this value to the real underlying sigma, which was 90 :)

(back to slides)

### Example: Variation of $\beta_0$ and $\beta_1$

Simulating **many** samples from the underlying known model

In [ ]:
# repeat with new sample:
y = beta_0 + beta_1*x + stats.norm.rvs(size = 20, loc=0, scale = sigma) # y-data with new values of residuals

# re-calculate parameters
Sxx = np.sum((x - x.mean())**2)
beta_1_hat = np.sum((x - x.mean())*(y - y.mean())) / Sxx
beta_0_hat = y.mean() - beta_1_hat*x.mean()
print(beta_0_hat, beta_1_hat)

#plot:
plt.scatter(x,y)
plt.plot([-2,4], [beta_0_hat + beta_1_hat*(-2), beta_0_hat + beta_1_hat*(4)], color="red")
plt.plot([-2,4], [beta_0 + beta_1*(-2), beta_0 + beta_1*(4)], color="black", linestyle='--')
plt.ylim([-400,1000])
plt.show()

Run the cell above several times

Observe how the estimated model changes due to fluctuations in data

Repeat for many samples and make histograms of estimated parameters:

In [ ]:
k = 1000
b0 = np.zeros(k)
b1 = np.zeros(k)
for i in range(k):
   y = beta_0 + beta_1*x + stats.norm.rvs(size = 20, loc=0, scale = sigma)
   beta_1_hat = np.sum((x - x.mean())*(y - y.mean())) / Sxx
   beta_0_hat = y.mean() - beta_1_hat*x.mean()
   b0[i] = beta_0_hat
   b1[i] = beta_1_hat

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,figsize=(8,4))

ax0.hist(b0)
ax0.axvline(x=beta_0, color="black", linestyle='--')

ax1.hist(b1)
ax1.axvline(x=beta_1, color="black", linestyle='--')

plt.show()


(back to slides)

### Example: Estimate standard errors of the parameters

In [ ]:
# Recall the height and weigth data:
print(student)

In [ ]:
### 1 ###  Estimate parameters beta_0_hat and bata_1_hat *manually*

Sxx = np.sum((student["x"] - student["x"].mean())**2)

beta_1_hat = np.sum((student["x"] - student["x"].mean())*(student["y"] - student["y"].mean())) / Sxx

beta_0_hat = student["y"].mean() - beta_1_hat*student["x"].mean()

print(beta_0_hat, beta_1_hat)

In [ ]:
# plot:
plt.scatter(student["x"], student["y"])  # plot the data
plt.plot([160, 200], [beta_0_hat + beta_1_hat*160, beta_0_hat + beta_1_hat*200], color="red")  # plot the line (from 160 to 200)
plt.show()

In [ ]:
### 2 ###  Estimate standard error for the parameters *manually* 

# first we add some columns to the dataset:
student["y_pred"] = beta_0_hat + beta_1_hat*student["x"]  # y-values ON the regression line
student["residuals"] = student["y"] - student["y_pred"]   # residuals
print(student)

In [ ]:
RSS = np.sum(student["residuals"]**2)
sigma_hat = np.sqrt(RSS/(10-2))

se_beta_0_hat = sigma_hat*np.sqrt(1/10 + student["x"].mean()**2 / Sxx)

se_beta_1_hat = sigma_hat*np.sqrt(1/Sxx)

print("Intercept: ", beta_0_hat, se_beta_0_hat)
print("slope:     ", beta_1_hat, se_beta_1_hat)

In [ ]:
### automatic ###  Do it all with inbuilt python function smf.ols
fitStudents = smf.ols(formula = 'y ~ x', data=student).fit()  # OBS: use the statsmodels.formula.api library (smf)
print(fitStudents.summary(slim=True))

In [ ]:
# Get parameters
print(fitStudents.params)

In [ ]:
# Get parameter standard errors
print(fitStudents.bse)

In [ ]:
# get estimate of sigma^2:
print(fitStudents.scale)
print(sigma_hat**2) #compare to the one we calculated *manualy*

In [ ]:
# get fitted values:
print(fitStudents.fittedvalues)

(compare with the "y_pred" values that we have computed above)

(back to slides)

### Example: Hypothesis test for parameters

In [ ]:
fitStudents = smf.ols(formula = 'y ~ x', data=student).fit()  # OBS: use the statsmodels.formula.api library (smf)
print(fitStudents.summary(slim=True))

In [ ]:
# Warning!! The p-values appear to be zero but they are in reality only smaller than 0.000

# print the pvalues seperately to be sure:
print(fitStudents.pvalues)

In [ ]:
# we can also do the test manually (here only for the intercept):

# calculate "t_obs"
t_obs_int = -119.9581/18.897
print(t_obs_int)

In [ ]:
# find corresponding p-value:
print(2*stats.t.cdf(t_obs_int, df=10-2))  # obs! df = n - 2  (2 is the number of parameters)

(back to slides)

### Example: Confidence interval for parameters

In [ ]:
fitStudents = smf.ols(formula = 'y ~ x', data=student).fit()  
print(fitStudents.summary(slim=True))

KAHOOT! (x1)

In [ ]:
# we can also calculate CI *manually*

# here only for the intercept:

print(-119.9581 - stats.t.ppf(0.975, df=10-2)*18.897)
print(-119.9581 + stats.t.ppf(0.975, df=10-2)*18.897)

(back to slides)

### Example: Confidence interval for the line

In [ ]:
# Recall data and model:
plt.scatter(student["x"], student["y"])  # plot the data
plt.plot([160, 200], [beta_0_hat + beta_1_hat*160, beta_0_hat + beta_1_hat*200], color="red")  # plot the line (from 160 to 200)
plt.show()

Lets make predictions of (mean) weight for some other height-values. 

What is the mean weight for persons of height 170cm, 175cm, 195cm or 205 cm?

We can calculate this manually (y = beta0_hat + beta1_hat * x)

But we can also use Python:

In [ ]:
# Make predictions (= mean y-value at a given x-value) and corresponding confidence intervals 
x_new = pd.DataFrame({'x': [170, 175, 195, 205]})

pred = fitStudents.get_prediction(x_new).summary_frame(alpha=0.05) 

print(pred.head())

The column "mean" is the predicted mean weight for each height in x_new

We also get a standard error for the mean weight (mean_se) and a conficence interval (for the specified alpha = 0.05)

In [ ]:
# Plot the new predictions of mean weight:

plt.scatter(student["x"], student["y"])
plt.plot(student["x"], student["y_pred"], color="red")
plt.scatter(x_new, pred["mean"], color="black")
plt.scatter(x_new, pred["mean_ci_lower"], color="black", marker="_")
plt.scatter(x_new, pred["mean_ci_upper"], color="black", marker="_")
plt.show()

Notice how the confidence interval i larger for more "extreme" heights

In [ ]:
# plot the confidence interval for large range of values:

x_new = pd.DataFrame({'x': np.arange(155, 205,0.1)})
pred = fitStudents.get_prediction(x_new).summary_frame(alpha=0.05) 

plt.scatter(student["x"],student["y"])
plt.plot(x_new, pred["mean"], color="red")
plt.plot(x_new, pred["mean_ci_lower"], color="grey")
plt.plot(x_new, pred["mean_ci_upper"], color="grey")
plt.show()

We could also calculate this interval *manually* (see Method 5.18)

(back to slides)

### Example: Prediction interval for observations

In [ ]:
# Same data as above

# now we want the prediction interval for individual (new/future) observations

print(pred.head())

In [ ]:
# plot the predictions intervals for a wide range of x-values:
plt.scatter(student["x"],student["y"])
plt.plot(x_new, pred["mean"], color="red")
plt.plot(x_new, pred["obs_ci_lower"], color="lightgrey")
plt.plot(x_new, pred["obs_ci_upper"], color="lightgrey")
plt.show()

In [ ]:
# Compare the prediction interval and the confidence interval
plt.scatter(student["x"],student["y"])
plt.plot(x_new, pred["mean"], color="red")
plt.plot(x_new, pred["obs_ci_lower"], color="lightgrey")
plt.plot(x_new, pred["obs_ci_upper"], color="lightgrey")
plt.plot(x_new, pred["mean_ci_lower"], color="grey")
plt.plot(x_new, pred["mean_ci_upper"], color="grey")
plt.show()

(back to slides)